In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Union

import candle
import experiments.textgenutils as gutils

## (1) Initialize Model with Pre-trained Weights

In [3]:
# One of ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
#    gpt2:         124M params
#    gpt2-medium:  354M params
#    gpt2-large:   774M params
#    gpt2-xl:    1,557M params

model = candle.models.gpt.GPT.from_pretrained('gpt2-large')

model.summary()

Layer Type # Parameters
                                                 
decoder_blocks      0   DecoderBlock     19677440
                    1   DecoderBlock     19677440
                    10  DecoderBlock     19677440
                    11  DecoderBlock     19677440
                    12  DecoderBlock     19677440
                    13  DecoderBlock     19677440
                    14  DecoderBlock     19677440
                    15  DecoderBlock     19677440
                    16  DecoderBlock     19677440
                    17  DecoderBlock     19677440
                    18  DecoderBlock     19677440
                    19  DecoderBlock     19677440
                    2   DecoderBlock     19677440
                    20  DecoderBlock     19677440
                    21  DecoderBlock     19677440
                    22  DecoderBlock     19677440
                    23  DecoderBlock     19677440
                    24  DecoderBlock     19677440
                    25  DecoderBlock     19677440
                    26  DecoderBlock     19677440
                    27  DecoderBlock     19677440
                    28  DecoderBlock     19677440
                    29  DecoderBlock     19677440
                    3   DecoderBlock     19677440
                    30  DecoderBlock     19677440
                    31  DecoderBlock     19677440
                    32  DecoderBlock     19677440
                    33  DecoderBlock     19677440
                    34  DecoderBlock     19677440
                    35  DecoderBlock     19677440
                    4   DecoderBlock     19677440
                    5   DecoderBlock     19677440
                    6   DecoderBlock     19677440
                    7   DecoderBlock     19677440
                    8   DecoderBlock     19677440
                    9   DecoderBlock     19677440
dropout                      Dropout            0
layer_norm                 LayerNorm         2560
output_projection          Parameter     64328960
position_embeddings        Embedding      1310720
word_embeddings            Embedding     64328960
Total                                   774030080

## (2) Have a conversation

In [11]:
def start_conversation(model,
                       name: str = 'John',
                       gpt_name: str = 'Taylor',
                       max_response_length: int = 50):
    print('|| ================================================================')
    print(f'|| Hello! You are now talking with {gpt_name} (send "bye" to exit)')
    print('|| ================================================================', end='')

    tokenizer = candle.models.gpt.GPT2BPETokenizer()
    chat_template = candle.nlp.SimpleConversationTemplate(user_name=name, assistant_name=gpt_name)
    model.clear_kv_cache()
    
    messages = []
    while True:
        print(f'\n\n[{name}] ', end='')

        prompt = input()
        messages.append({'role': 'user', 'content': prompt})

        if prompt.lower() == 'bye':
            break

        chat = chat_template.apply_chat_template(messages, add_generation_prompt=True, n_recent_messages=2)

        print(f'\n[{gpt_name}] ', end='')
        response = gutils.display_stream_of_text(
            model,
            tokenizer,
            prompt=chat,
            n_tokens_to_generate=max_response_length,
            beam_size=1,
            top_k=100,
            top_p=0.98,
            stop_strings={'\n': 1, '<|endoftext|>': 1, '\.|\!|\?': 4},
            use_kv_cache=True
        )
        messages.append({'role': 'assistant', 'content': response})

In [12]:
start_conversation(model, name='John')

|| ================================================================
|| Hello! You are now talking with Taylor (send "bye" to exit)
|| ================================================================

[John] Hi Taylor!

[Taylor] You look like a beautiful human. That outfit is cute.

[John] Wow thanks!

[Taylor] But you look a lot prettier dressed up in that shirt, that skirt, and black pants. That's more than I could say for myself! Thanks for the shoutout and all the compliments!

[John] Uh... wow thanks! 

[Taylor] Did you know that I'm an escort that does webcam shows?

[John] I did not know that

[Taylor] Awesome!

[John] Bye


In [13]:
start_conversation(model, name='John')

|| ================================================================
|| Hello! You are now talking with Taylor (send "bye" to exit)
|| ================================================================

[John] Hi Taylor

[Taylor] Hey John, I was wondering if I could borrow that fanny pack, do you have anyone who has one?

[John] Sure, my friend Bob has one.

[Taylor] All right.

[John] Want it now?

[Taylor] Sure, sure...

[John] OK! Let me get it for you

[Taylor] Sure! Thank you!

[John] Bye


In [ ]:
start_conversation(model, name='John')

|| ================================================================
|| Hello! You are now talking with Taylor (send "bye" to exit)
|| ================================================================

[John] Hi Taylor!

[Taylor] What's up?

[John] Where are you from?

[Taylor] San Diego

[John] Wow I love it there

[Taylor] What kind of football team do you like?

[John] I like the kind that wins but not too much

[Taylor] That sucks. I was gonna say the type where your quarterback's main job is to defend from everyone by throwing balls and running people around?

[John] Nice that's cool too

[Taylor] Anyway.. What city did you go to high school in before?

[John] Bellevue, Washington

[Taylor] How long ago were you there?

[John] 10 years ago, give or take?

[Taylor] No.

[John] 

In [16]:
start_conversation(model, name='John')

|| ================================================================
|| Hello! You are now talking with Taylor (send "bye" to exit)
|| ================================================================

[John] Hi!

[Taylor] I was wondering which of our friends in the UK would come over soon, what have you got planned for Christmas.

[John] I was going to home to see my family. How about you?

[Taylor] Yup! I will be up on the 5th, then off to Dublin to see my mom before I go back in to Toronto with my friends.

[John] That's great. Hope she's doing well

[Taylor] Yeah!

[John] Well see you later!

[Taylor] OK.

[John] Bye
